## Xview Dataset clipping
This is a component designed to clip the dataset provided by xview. 

In [3]:
!pip install Pillow

In [2]:

import os
from PIL import Image


In [4]:

#directory_path is the path to the folder with the unzipped .tif images from xview dataset 
directory_path = os.environ.get("directory_path")

#destination is the path to the folder which saves all the extracted tiles. 
destination = os.environ.get("destination")

#Each image is cropped using a rectangular window with edge_length tile_size_x and tile_size_y which has to be given in number of pixels 
tile_size_x = os.environ.get("tile_size_x")
tile_size_y = os.environ.get("tile_size_y")

#stride_x is the length in pixels the sliding window is moved to the right after each step
#stride_y is the length in pixels the sliding window is moved down after completing a row
#For tumbling window stride_x must equal tile_size_x and stride_y must equal tile_size_y
stride_x = os.environ.get("stride_x")
stride_y = os.environ.get("stride_y")


In [5]:

def clear_directory(directory):
    for item in os.listdir(directory):
            item_path = os.path.join(directory, item)
            # Check if the item is a file
            if os.path.isfile(item_path):
                # If it's a file, remove it
                os.remove(item_path)
            # If it's a directory, recursively clear it
            elif os.path.isdir(item_path):
                clear_directory(item_path)
                os.rmdir(item_path)
                

In [9]:

def sliding_window(directory_path, destination, tile_size_x, tile_size_y, stride_x, stride_y):
    
    if os.path.isdir(destination):
        clear_directory(destination)
    else: 
        os.makedirs(destination)
        
    
    for item in os.listdir(directory_path):
        if item == ".DS_Store":
            continue
        item_path = os.path.join(directory_path, item)
        clipped_item = os.path.join(destination, item)
        os.makedirs(clipped_item)
        
        image = Image.open(item_path)
        width, height = image.size
        
        x_range = [0]
        while(x_range[-1] + stride_x + tile_size_x < width):
            x_range += [x_range[-1] + stride_x]
        
        y_range = [0]
        while(y_range[-1] + stride_y + tile_size_y < height):
            y_range += [y_range[-1] + stride_y]
        
        counter = 0
        for x in x_range:
            for y in y_range:
                cropped = image.crop((x,y, x+tile_size_x, y+tile_size_y))

                file_name = "{}".format(y)
                file_name += "_{}".format(x) + ".tif"
                path = os.path.join(clipped_item, file_name)
                
                cropped.save(path)
                


In [10]:
sliding_window(directory_path, destination, tile_size_x, tile_size_y, stride_x, stride_y)

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType